In [6]:
from keys import DEEP_SEEK_TEST_KEY  ## Way of concealing api key form VCS like git
from langchain.llms import OpenAI  ## Simpler way of just getting paragraph responses against singular prompts with no past memory
from langchain_openai import ChatOpenAI ## Better method for seding sofisticated prompts and best suited for chatbots, RAG etc, because of past memory of the conversation
import os ## for setting environment variables

### Setting environment variables
They are basically api keys of the llm models.

In [2]:
os.environ["OPENAI_API_KEY"] = DEEP_SEEK_TEST_KEY
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1" # Since we are using deepseek free version here we have to use openrouter.ai platform and for that we need to set this environment variable

### Initializing the model and testing

In [7]:
llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3-0324:free",  # Choose a DeepSeek model
    temperature=0.7, # This is a metric which sets how random and creative answers the llm gives (0-1 range)
)

In [ ]:
response = llm.invoke(f"You are an english teacher. Consider the following conversation between two of your students and access in which areas of grammer they are lagging behind: {conversation} \n also tell which of them is lagging behind some quick topics for them to study on")

print(response)